# This notebook is used for modelling between the livability of melbourne and rental affordability

For the sake of simplicty, this notebook contains the following
- Livability scores of melbourne
- Rental affordability of greater Melbourne

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm

## Data processing

The below data will be used for this model:

### Melbourne livability score

- This dataset contains multiple liveability related scores for multiple regions

### Predictor variable: Greater Melbourne rental affordability index

- This dataset contains information about the rental affordability 

## Initial data analysis

In [51]:
# Load data

liveability = pd.read_csv("../data/predictor/Melbourne Liveability Indicators.csv")
rental_affordability = pd.read_csv("../data/response/rental_affordability.csv")
rental_affordability.columns = rental_affordability.columns.str.replace(' ', '')

In [4]:
liveability.columns.sort_values()

Index(['denominator', 'id', 'indicator', 'numerator', 'period', 'sources',
       'topic', 'type', 'value', 'value_type'],
      dtype='object')

In [5]:
set(liveability['period'])

{'2006',
 '2011',
 '2012',
 '2013',
 '2013/14',
 '2014',
 '2014 (June Qtr)',
 '2014 (at 30 June)',
 '2014/15',
 '2015',
 '2015 (June Qtr)',
 '2015 (at 30 June)',
 '2015/16',
 '2016',
 '2016 (June Qtr)',
 '2016 (at 30 June)',
 '2016/17',
 '2017',
 '2017 (June Qtr)',
 '2017 (at 30 June)',
 '2018',
 '2018 (June Qtr)',
 '2018 (at 30 June)',
 '2019',
 'FY 2013/14',
 'FY 2014/15',
 'FY 2015/16',
 'FY 2016/17',
 'FY 2017/18',
 'FY 202013/14',
 'FY 202014/15',
 'FY 202015/16'}

For the liveability dataset, we have information from 2006, 2011~2019. Now we proceed to look at rental_affordability

In [6]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3005,NaN,NaN,NaN,NaN,VIC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
5,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


In [7]:
rental_affordability.columns.sort_values()

Index(['city', 'geography_name', 'rai_cityadjusted_total_2011_q1',
       'rai_cityadjusted_total_2011_q2', 'rai_cityadjusted_total_2011_q3',
       'rai_cityadjusted_total_2011_q4', 'rai_cityadjusted_total_2012_q1',
       'rai_cityadjusted_total_2012_q2', 'rai_cityadjusted_total_2012_q3',
       'rai_cityadjusted_total_2012_q4', 'rai_cityadjusted_total_2013_q1',
       'rai_cityadjusted_total_2013_q2', 'rai_cityadjusted_total_2013_q3',
       'rai_cityadjusted_total_2013_q4', 'rai_cityadjusted_total_2014_q1',
       'rai_cityadjusted_total_2014_q2', 'rai_cityadjusted_total_2014_q3',
       'rai_cityadjusted_total_2014_q4', 'rai_cityadjusted_total_2015_q1',
       'rai_cityadjusted_total_2015_q2', 'rai_cityadjusted_total_2015_q3',
       'rai_cityadjusted_total_2015_q4', 'rai_cityadjusted_total_2016_q1',
       'rai_cityadjusted_total_2016_q2', 'rai_cityadjusted_total_2016_q3',
       'rai_cityadjusted_total_2016_q4', 'rai_cityadjusted_total_2017_q1',
       'rai_cityadjusted_total_20

From a quick look , it looks like geography_name=3005 seems to only hold NaN values, and thus can get rid of it

In [8]:
rental_affordability[rental_affordability['geography_name']==3005]

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
3,3005,NaN,NaN,NaN,NaN,VIC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
rental_affordability[['unique_id', 'city', 'geography_name', 'state']]

,unique_id,city,geography_name,state
0,1310,Greater Melbourne,3000,VIC
1,1312,Greater Melbourne,3003,VIC
2,1313,Greater Melbourne,3004,VIC
3,1314,Greater Melbourne,3005,VIC
4,1315,Greater Melbourne,3006,VIC
5,1316,Greater Melbourne,3008,VIC


From the above information, we can remove all columns but geography_name (which seems to be postcode), and also remove where geography_name=3005

In [10]:
rental_affordability = rental_affordability.drop(3)
rental_affordability = rental_affordability.reset_index()
rental_affordability.drop(columns=rental_affordability.columns[0], axis=1, inplace=True)

## Dataset cleanup
Now that we have the appropriate information, we finalise the dataset so that we can use for modelliing

In [12]:
liveability

,type,topic,id,indicator,period,numerator,denominator,value,value_type,sources
0,Liveability,Economy,ECO_1,City's unemployment rate,2015 (June Qtr),"3,942","84,942",4.64,Percentage,Australian Government Department of Employment...
1,Liveability,Economy,ECO_2,Assessed value of commercial and industrial pr...,2015 (at 30 June),"48,331,069,777.00","101,671,404,180.00",47.54,Percentage,"City of Melbourne, Property Services, 2015-2018"
2,Liveability,Economy,ECO_3,Percentage of city population living in poverty,2015,"13,177","136,872",9.63,Percentage,"Geografia, Relative Poverty and Employment Mea..."
3,Liveability,Economy,ECO_4,Percentage of persons (city population) in ful...,2017 (June Qtr),"68,722","159,141",43.18,Percentage,"Geografia, Relative Poverty and Employment Mea..."
4,Liveability,Economy,ECO_4,Percentage of persons (city population) in ful...,2014 (June Qtr),"55,359","127,975",43.26,Percentage,"Geografia, Relative Poverty and Employment Mea..."
...,...,...,...,...,...,...,...,...,...,...
314,Social,Participation in activities,ACT_7,Percentage of residents who have participated ...,2019,NaN,"1,259",49.30,Percentage,City of Melbourne Social Indicators Survey 2019
315,Social,"Culture, diversity and safety",CULT_4,Percentage of residents that reported feeling ...,2019,NaN,"1,235",57.40,Percentage,City of Melbourne Social Indicators Survey 2019
316,Social,"Culture, diversity and safety",CULT_5,Percentage of residents that reported feeling ...,2019,NaN,"1,197",86.00,Percentage,City of Melbourne Social Indicators Survey 2019
317,Social,"Culture, diversity and safety",CULT_5,Percentage of residents that reported feeling ...,2018,NaN,"1,188",86.50,Percentage,City of Melbourne Social Indicators Survey 2018


In [11]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
4,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


Firstly, we need to be able to condense both of the datasets into year so that we can model easier. For the liveability score, we'll combine numerator + denominator for a whole year, then calculate a new percentage for said year. Further, we only want percentages for our dataset since counts will not give us any relevant predictions, as we do not need to create a model but rather see some correlation between the 2 datasets

In [77]:
# Get the subset of the dataframe with the data we want

liveability = liveability[liveability['type'] == 'Liveability']
liveability = liveability[liveability['value_type'] == 'Percentage']
liveability['period'] = liveability['period'].str.extract(r'(\d{4})')
set(liveability['topic'])

{'Economy',
 'Finance',
 'Governance',
 'Profile - housing, government and economy',
 'Profile - people',
 'Transportation'}

In [78]:
# Then remove all NaN values

liveability = liveability.dropna(subset=['numerator', 'denominator'])

In [81]:
# Turn all numerical columns into int for calculation

liveability['numerator'] = liveability['numerator'].astype(float).astype(int)
liveability['denominator'] = liveability['denominator'].str.replace(',', '').astype(float).astype(int)

In [87]:
liveability_clean = liveability[['period', 'numerator', 'denominator']].groupby('period').sum().reset_index()
liveability_clean['percentage'] = liveability_clean['numerator']/liveability_clean['denominator']

In [88]:
liveability_clean

,period,numerator,denominator,percentage
0,2006,244247,477011,0.512036
1,2011,318057,601122,0.529106
2,2012,173509,193858,0.895031
3,2013,1464955487258,83367118711,17.572342
4,2014,1349723751742,71080444615,18.988679
5,2015,1257094727637,171660973342,7.323125
6,2016,1379316522257,188469803517,7.318501
7,2017,77460334182,161875660309,0.478517
8,2018,83219880059,183434791145,0.453676


## Data modelling

In [198]:
partial_model = ols(
    formula='population ~ building_permit_count',
    data=analysis_df
).fit()

print(partial_model.summary())

                            OLS Regression Results                            
Dep. Variable:             population   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     61.04
Date:                Thu, 16 May 2024   Prob (F-statistic):           2.38e-07
Time:                        12:03:06   Log-Likelihood:                -237.49
No. Observations:                  21   AIC:                             479.0
Df Residuals:                      19   BIC:                             481.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              8.575e+

In [199]:
full_model = ols(
    formula='population ~ building_permit_count + year',
    data=analysis_df
).fit()

print(full_model.summary())

                            OLS Regression Results                            
Dep. Variable:             population   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     429.4
Date:                Thu, 16 May 2024   Prob (F-statistic):           6.47e-16
Time:                        12:03:06   Log-Likelihood:                -211.79
No. Observations:                  21   AIC:                             429.6
Df Residuals:                      18   BIC:                             432.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -1.074e+

It appears that adding the year as a variable after having building permit counts is still statistically significant. We also test for interaction

In [201]:
interaction_model = ols(
    formula='population ~ building_permit_count + year + building_permit_count*year',
    data=analysis_df
).fit()

print(interaction_model.summary())

                            OLS Regression Results                            
Dep. Variable:             population   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     500.4
Date:                Thu, 16 May 2024   Prob (F-statistic):           8.93e-17
Time:                        12:09:38   Log-Likelihood:                -205.43
No. Observations:                  21   AIC:                             418.9
Df Residuals:                      17   BIC:                             423.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

Interaction also seems to be significant, even with the other 2 features already being added to the model. Test for goodness of fit

In [202]:
interaction_model.aic < full_model.aic

True

In [203]:
anova_lm(interaction_model)

,df,sum_sq,mean_sq,F,PR(>F)
building_permit_count,1.0,2.627851e+10,2.627851e+10,1157.817349,4.438442e-17
year,1.0,7.472806e+09,7.472806e+09,329.247827,1.459208e-12
building_permit_count:year,1.0,3.215219e+08,3.215219e+08,14.166084,1.547890e-03
Residual,17.0,3.858422e+08,2.269660e+07,NaN,NaN


Attempt to predict 2022, 2023 and 2024 population

In [207]:
predict_df = predictor_clean.iloc[21:]

In [208]:
interaction_model.predict(predict_df)

22    194064.006211
23    201738.853351
24    175977.057089
dtype: float64

Round the numbers to the nearest whole. Thus, our final conclusion can be:

### 2022

194064 people

### 2023

201739 people

### 2024

175977 people